<font color=gray>ADS Sample Notebook.

Copyright (c) 2019,2020 Oracle, Inc.  All rights reserved.
Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
</font>

***
# <font color=red>Develop pyspark jobs locally - from Local to Remote Workflows</font>
<p style="margin-left:10%; margin-right:10%;">by the <font color=teal> Oracle Cloud Infrastructure Data Science Team </font></p>

***

## Overview of this Notebook

This notebook provides spark operations for customers by bridging the existing local spark workflows with cloud based capabilities. Data scientists can use their familiar local environments with JupyterLab and work with remote data and remote clusters simply by selecting a kernel. The operations that will be demonstrated are: how to use the interactive spark env and product a spark script, how to prepare and create an application; how to prepare and create a run; how to list existing dataflow applications; and how to retrieve and display the logs.

The purpose of the dataflow module is to provide an efficient and convenient way for users to launch a spark application and run spark jobs. The interactive spark kernel provides a simple and efficient way to edit and build your spark script, and easy access to read from oci filesystem.


<a id='prereq'></a>
## Prerequisites:

1. Before accessing oci filesystem from your local spark env, please make sure that you have the `core-site.xml` under `spark_conf_dir` configured properly, because it sets the connector properties that we use to connect to oci.

2. Before creating applications in the Oracle Cloud Infrastructure Data Flow service, make sure that you have configured your tenancy for the service. Please follow the steps listed in this section of the Data Flow documentation: 


https://docs.cloud.oracle.com/en-us/iaas/data-flow/using/dfs_getting_started.htm#getting_started

In [ ]:
import io
import matplotlib.pyplot as plt
import os
from os import path
import pandas as pd
import tempfile
import uuid

from ads.dataflow.dataflow import DataFlow

from pyspark.sql import SparkSession

<a id='kernel'></a>
### Build your pyspark script using interactive spark kernel 

Set up spark session in your pyspark conda env

In [ ]:
# create a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.driver.cores", "4") \
    .config("spark.executor.cores", "4") \
    .getOrCreate()

Load the Employee Attrition data file from Oracle Cloud Infrastructure Object Storage into a Spark DataFrame

In [ ]:
emp_attrition = spark\
      .read\
      .format("csv")\
      .option("header", "true")\
      .option("inferSchema", "true")\
      .option("multiLine", "true")\
      .load("oci://hosted-ds-datasets@bigdatadatasciencelarge/synthetic/orcl_attrition.csv") \
      .cache() # cache the dataset to increase computing speed
emp_attrition.createOrReplaceTempView("emp_attrition")

Explore the dataframe

In [ ]:
spark.sql('select * from emp_attrition limit 5').toPandas()

Visualize how monthly income and age relate to one another in the context of years in industry

In [ ]:
fig, ax = plt.subplots() 
plot = spark.sql("""
          SELECT 
              Age,
              MonthlyIncome,
              YearsInIndustry
          FROM
            emp_attrition 
          """).toPandas().plot.scatter(x="Age", y="MonthlyIncome", title='Age vs Monthly Income',
                                       c="YearsInIndustry", cmap="viridis", figsize=(12,12), ax=ax)
plot.set_xlabel("Age")
plot.set_ylabel("Monthly Income")
plot

View all of the columns in the table

In [ ]:
spark.sql("show columns from emp_attrition").show()

Select a few columns using spark and convert it into a pandas DataFrame

In [ ]:
df = spark.sql("""
         SELECT
            Age,
            MonthlyIncome,
            YearsInIndustry
          FROM
            emp_attrition """).limit(10).toPandas()
df

We can also work with different compression formats within Dataflow. For example snappy parquet: 

In [ ]:
# Writing to a snappy parquet file
df.to_parquet('emp_attrition.parquet.snappy', compression='snappy')
pd.read_parquet('emp_attrition.parquet.snappy')  

In [ ]:
# We are able to read in this snappy parquet file to a spark dataframe
read_snappy_df = SparkSession \
    .builder \
    .appName("Snappy Compression Loading Example") \
    .config("spark.io.compression.codec", "org.apache.spark.io.SnappyCompressionCodec") \
    .getOrCreate() \
    .read \
    .format("parquet") \
    .load(f"{os.getcwd()}/emp_attrition.parquet.snappy")

read_snappy_df.first()

Note: other compression formats Data Flow supports today include snappy parquet (example above) and gzip on both csv and parquet.

We have come to a query that we want to run in Data Flow from previous explorations. Please refer to the dataflow.ipynb on how to submit a job to dataflow

In [ ]:
dataflow_base_folder = tempfile.mkdtemp()
data_flow = DataFlow(dataflow_base_folder=dataflow_base_folder)
print("Data flow directory: {}".format(dataflow_base_folder))

In [ ]:
pyspark_file_path = path.join(dataflow_base_folder, "example-{}.py".format(str(uuid.uuid4())[-6:]))
script = '''
from pyspark.sql import SparkSession

def main():
    
    # Create a Spark session
    spark = SparkSession \\
        .builder \\
        .appName("Python Spark SQL basic example") \\
        .getOrCreate()
    
    # Load a csv file from dataflow public storage
    df = spark \\
        .read \\
        .format("csv") \\
        .option("header", "true") \\
        .option("multiLine", "true") \\
        .load("oci://hosted-ds-datasets@bigdatadatasciencelarge/synthetic/orcl_attrition.csv")
    
    # Create a temp view and do some SQL operations
    df.createOrReplaceTempView("emp_attrition")
    query_result_df = spark.sql("""
        SELECT 
            Age,
            MonthlyIncome,
            YearsInIndustry
        FROM emp_attrition 
    """)
    
    # Convert the filtered Spark DataFrame into JSON format
    # Note: we are writing to the spark stdout log so that we can retrieve the log later at the end of the notebook.
    print('\\n'.join(query_result_df.toJSON().collect()))
    
if __name__ == '__main__':
    main()
'''

with open(pyspark_file_path, 'w') as f:
    print(script.strip(), file=f)
    
print("Script path: {}".format(pyspark_file_path))

In [ ]:
script_bucket = "test"                     # Update the value
logs_bucket = "dataflow-log"               # Update the value
display_name = "sample_Data_Flow_app"      

app_config = data_flow.prepare_app(display_name=display_name,
                                   script_bucket=script_bucket,
                                   pyspark_file_path=pyspark_file_path,
                                   logs_bucket=logs_bucket)

app = data_flow.create_app(app_config)

run_display_name = "sample_Data_Flow_run"
run_config = app.prepare_run(run_display_name=run_display_name)

run = app.run(run_config, save_log_to_local=True)

In [ ]:
run.status

In [ ]:
run.config

In [ ]:
run.oci_link